In [1]:
from jbfg.pandas_utils import *
import matplotlib.pyplot as plt
import seaborn as sns
import calendar
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
import plotly
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [2]:
custData = pd.read_csv("../0.data/bank_churner.csv")
custData[['cstno', 'tot_product_count', 'is_churned']]

,cstno,tot_product_count,is_churned
0,708082083,4,0
1,708083283,3,1
2,708084558,6,1
3,708085458,6,0
4,708086958,3,0
...,...,...,...
8096,828285858,3,0
8097,828288333,4,0
8098,828294933,3,1
8099,828298908,4,0


In [18]:
# 상품건수와 이탈 여부 사이 관계

sql = f"""

 SELECT 
    CSTNO     
    ,tot_product_count -- 현재 보유 상품 개수        
    ,is_churned   -- 이탈여부
  FROM l2_kjbk_extnl.bank_churner 
    
"""

In [3]:
# custData = get_df(sql)
for index, row in custData.iterrows() :
    for line in custData :
        custData.at[index,line] = converDataFormat(line, row[line])    

NameError: name 'converDataFormat' is not defined

In [36]:
# 현재 보유 상품 개수 범위 추출

In [11]:
# 상품건수와 이탈 여부 사이 관계

sql = f"""

 SELECT 
    tot_product_count
  FROM l2_kjbk_extnl.bank_churner 
  group by tot_product_count
    
"""

In [12]:
custData = get_df(sql)
for index, row in custData.iterrows() :
    for line in custData :
        custData.at[index,line] = converDataFormat(line, row[line])    
        productCount = custData["tot_product_count"]
        
product_min = custData.min()
product_max = custData.max()

In [34]:
# 현재 보유 상품 개수 중 이탈 여부가 높은 구간 추출

In [13]:
# 이탈 여부가 높은 구간 추출

sql = f"""

 SELECT 
    COUNT(*)
  FROM l2_kjbk_extnl.bank_churner 
  WHERE is_churned = 'Y'
  AND tot_product_count = '${productCount}'
    
"""

In [14]:
custData = get_df(sql)
for index, row in custData.iterrows() :
    for line in custData :
        custData.at[index,line] = converDataFormat(line, row[line])    
print(custData)

   expr_1
0       0


In [ ]:
# 함수 모음

In [8]:
#데이터 포맷 가공 함수
def converDataFormat(col, param) :
    #기본 데이터 NULL, "", Unknown 은 0으로 처리
    if param is None :
      param =  0
    if param == "" :
      param =  0
    if param == "Unknown":
      param =  0    
    # 기본 데이터 끝    
    
    return param
    

    

In [20]:
# 함수 모음 끝

In [20]:
#특성과 타켓 변수 분리
custData = custData.dropna()
X = custData[["tot_product_count"]]
Y = custData["is_churned"]
#학습 데이터와 테스트 데이터 분할
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2, random_state=42)
model = LogisticRegression()
#로지스틱 회귀 모델 학습
model.fit(X_train, Y_train)
#예측
Y_pred = model.predict(X_test)
#정확도 평가
accuracy = accuracy_score(Y_test,Y_pred)
print("Accuracy:",accuracy)

Accuracy: 0.8285009253547193


In [21]:
# 소진율과 이탈 여부 사이 관계

sql = f"""

 SELECT 
    CSTNO     
    ,mean_open_to_buy -- 평균 사용가능 신용한도
    ,mean_util_pct -- 평균 한도 소진율        
    , months_inact_for_12m -- 거래가 없었던 개월 수
    ,is_churned   -- 이탈여부
  FROM l2_kjbk_extnl.bank_churner 
    
"""

In [22]:
custData = get_df(sql)
for index, row in custData.iterrows() :
    for line in custData :
        custData.at[index,line] = converDataFormat(line, row[line])    

In [24]:
#특성과 타켓 변수 분리
custData = custData.dropna()
X = custData[["mean_open_to_buy","mean_util_pct","months_inact_for_12m"]]
Y = custData["is_churned"]
#학습 데이터와 테스트 데이터 분할
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2, random_state=42)
model = LogisticRegression()
#로지스틱 회귀 모델 학습
model.fit(X_train, Y_train)
#예측
Y_pred = model.predict(X_test)
#정확도 평가
accuracy = accuracy_score(Y_test,Y_pred)
print("Accuracy:",accuracy)

Accuracy: 0.8223318938926588


In [27]:
# 카드 거래가 없는 개월수와 이탈 여부 사이 관계

sql = f"""

 SELECT 
    CSTNO         
    , months_inact_for_12m -- 거래가 없었던 개월 수
    ,is_churned   -- 이탈여부
  FROM l2_kjbk_extnl.bank_churner 
    
"""

In [28]:
custData = get_df(sql)
for index, row in custData.iterrows() :
    for line in custData :
        custData.at[index,line] = converDataFormat(line, row[line])    

In [30]:
#특성과 타켓 변수 분리
custData = custData.dropna()
X = custData[["months_inact_for_12m"]]
Y = custData["is_churned"]
#학습 데이터와 테스트 데이터 분할
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2, random_state=42)
model = LogisticRegression()
#로지스틱 회귀 모델 학습
model.fit(X_train, Y_train)
#예측
Y_pred = model.predict(X_test)
#정확도 평가
accuracy = accuracy_score(Y_test,Y_pred)
print("Accuracy:",accuracy)

Accuracy: 0.8285009253547193
